In [1]:
from __future__ import annotations

from etils.lazy_imports import *
import notion_client

import common

In [2]:
common.add_notion_to_sys_path()

In [3]:
ecolab.clear_cached_modules(['auto_notion'])

import auto_notion

In [4]:
db = auto_notion.Database('2f1baf0d3c114887ab7a32912a235589')
db

In [5]:
all_rows = list(db)
all_rows = sorted(all_rows, key=lambda r: r.created_time)

In [6]:
def to_list(values: str | None, *, process_fn = None) -> list[int]:
    process_fn = process_fn or (lambda x: x)
    if values is None:
        return []
    elif not values.strip():
        return []
    results = []
    for val in values.split(','):
        if not val:
            continue
        elif val.endswith('*'):
            val = val.rstrip('*')
            val = int(val) / 2
        else:
            val = int(val)
        val = process_fn(val)
        results.append(val)
    return results

In [37]:
ecolab.clear_cached_modules(['plotly_dc'])

import plotly_dc


@dataclasses.dataclass
class Session:
    sets: list[float]
    date: datetime.datetime

    @property
    def max(self) -> float:
        return max(self.sets, default=0)

    @property
    def total(self) -> float:
        return sum(self.sets)


@dataclasses.dataclass
class Excercise(plotly_dc.Visualizable):
    sessions: list[Session]

    @property
    def dates(self):
        return [sess.date for sess in self.sessions if sess.total]

    @property
    def max(self):
        return max(self.maxs)

    @property
    def max_total(self):
        return max(self.maxs)

    @property
    def maxs(self):
        return [sess.max for sess in self.sessions if sess.total]

    @property
    def totals(self):
        return [sess.total for sess in self.sessions if sess.total]

    @property
    def zeros(self):
        return [0 for sess in self.sessions if sess.total]
    
    @property
    def cum_max(self):
        curr = 0
        cums = []
        for v in self.maxs:
            if v > curr:
                curr = v
            cums.append(curr)
        return cums
    
    @property
    def cum_total(self):
        curr = 0
        cums = []
        for v in self.totals:
            if v > curr:
                curr = v
            cums.append(curr)
        return cums
    
    # TODO: Weekly agregate
    @property
    def week_df(self):
        # Warning: Mean can contain 0 values
        return self.df.groupby(pd.Grouper(key='dates', freq='W-MON', label='left', closed='left')).agg(
            maxs_of_the_week=pd.NamedAgg(column="maxs", aggfunc="max"),
            weekly_volume=pd.NamedAgg(column="totals", aggfunc="sum"),
            zeros=pd.NamedAgg(column="zeros", aggfunc="sum"),
            num_sess=("dates", "size"),
        )

    Y_SESSION = [
        'maxs',
        'totals',
        # 'cum_total',
        # 'cum_max',
        'zeros',
    ]
    _df_fields = [
        'dates',
    ] + Y_SESSION


def get_metric(excercise_name, process_fn=None):
    return Excercise([Session(
        sets=to_list(getattr(row, excercise_name), process_fn=process_fn),
        date=row.created_time,
    ) for row in all_rows])


def _num_to_sec(x):
    num_min = x // 100
    num_sec = x % 100
    return num_min * 60 + num_sec


pull_up = get_metric('pull_up')
push_up = get_metric('push_up')
planche = get_metric('planche', process_fn=_num_to_sec)

## Weekly

In [38]:
px.line(push_up.week_df, y=['maxs_of_the_week', 'weekly_volume', 'zeros', 'num_sess'])

In [40]:
px.line(pull_up.week_df, y=['maxs_of_the_week', 'weekly_volume', 'zeros', 'num_sess'])

In [41]:
px.line(planche.week_df, y=['maxs_of_the_week', 'weekly_volume', 'zeros'])

## Session

In [12]:
px.line(push_up.df, x='dates', y=pull_up.Y_SESSION)

In [13]:
px.line(planche.df, x='dates', y=pull_up.Y_SESSION)

In [14]:
for row in all_rows:
    vals = to_list(row.pull_up)
    if vals and not row.pull_total:
        row.pull_total = sum(vals)

    vals = to_list(row.push_up)
    if vals and not row.push_total:
        row.push_total = sum(vals)